In [1]:
# -*- coding: utf-8 -*-
import argparse
import numpy as np
import time
import random
import torch
import torch.nn.functional as F
import dgl
from gat import GAT

from utils import load_network_data, get_train_data, random_planetoid_splits
from loss import multihead_contrastive_loss
import warnings
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser(description='GAT')
parser.add_argument("--gpu", type=int, default=0,
                    help="which GPU to use. Set -1 to use CPU.")
parser.add_argument("--epochs", type=int, default=2000,
                    help="number of training epochs")
parser.add_argument("--dataset", type=str, default="citeseer",
                    help="which dataset for training")
parser.add_argument("--num-heads", type=int, default=4,
                    help="number of hidden attention heads")
parser.add_argument("--num-layers", type=int, default=1,
                    help="number of hidden layers")
parser.add_argument("--num-hidden", type=int, default=32,
                    help="number of hidden units")
parser.add_argument("--tau", type=float, default=1,
                    help="temperature-scales")
parser.add_argument("--seed", type=int, default=1,
                    help="random seed")
parser.add_argument("--in-drop", type=float, default=0.6,
                    help="input feature dropout")
parser.add_argument("--attn-drop", type=float, default=0.5,
                    help="attention dropout")
parser.add_argument("--lr", type=float, default=0.01,
                    help="learning rate")
parser.add_argument('--weight-decay', type=float, default=1e-4,
                    help="weight decay")
parser.add_argument('--negative-slope', type=float, default=0.2,
                    help="the negative slope of leaky relu")

args = parser.parse_known_args()[0]
print(args)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

adj2, features, Y = load_network_data(args.dataset)
features[features > 0] = 1
g = dgl.from_scipy(adj2)

if args.gpu >= 0 and torch.cuda.is_available():
    cuda = True
    g = g.int().to(args.gpu)
else:
    cuda = False

features = torch.FloatTensor(features.todense())
f = open('NCLA_' + args.dataset + '.txt', 'a+')
f.write('\n\n\n{}\n'.format(args))
f.flush()

labels = np.argmax(Y, 1)
adj = torch.tensor(adj2.todense())

all_time = time.time()
num_feats = features.shape[1]
n_classes = Y.shape[1]
n_edges = g.number_of_edges()

# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# create model
heads = ([args.num_heads] * args.num_layers)
model = GAT(g,
            args.num_layers,
            num_feats,
            args.num_hidden,
            heads,
            F.elu,
            args.in_drop,
            args.attn_drop,
            args.negative_slope)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()

# use optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# initialize graph
dur = []
test_acc = 0

counter = 0
min_train_loss = 100
early_stop_counter = 100
best_t = -1

for epoch in range(args.epochs): #按照 args.epochs 指定的次数进行多轮训练。
    if epoch >= 0:
        t0 = time.time()   #计时
    model.train() #模型设置为训练模式
    optimizer.zero_grad() #梯度清零
    heads = model(features)#然后进行前向传播以获得模型预测
    loss = multihead_contrastive_loss(heads, adj, tau=args.tau)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        heads = model(features)
        loss_train = multihead_contrastive_loss(heads, adj, tau=args.tau)

    #如果训练损失连续 100 轮都没有下降，则进行提前停止
    if loss_train < min_train_loss:
        counter = 0
        min_train_loss = loss_train
        best_t = epoch
        torch.save(model.state_dict(), 'best_NCLA.pkl')
    else:
        counter += 1

    if counter >= early_stop_counter:
        print('early stop')
        break

    if epoch >= 0:
        dur.append(time.time() - t0)

    print("Epoch {:04d} | Time(s) {:.4f} | TrainLoss {:.4f} ".
          format(epoch + 1, np.mean(dur), loss_train.item()))

print('Loading {}th epoch'.format(best_t))
model.load_state_dict(torch.load('best_NCLA.pkl'))#训练结束后，从最佳轮次加载模型参数 
model.eval()#将模型设置为评估模式，并通过模型传递特征以获得嵌入
with torch.no_grad():
    heads = model(features)
embeds = torch.cat(heads, axis=1)  #从所有头部中串联嵌入
embeds = embeds.detach().cpu()

Accuaracy_test_allK = []
numRandom = 20

for train_num in [1, 2, 3, 4, 20]:

    AccuaracyAll = []
    for random_state in range(numRandom):
        print(
            "\n=============================%d-th random split with training num %d============================="
            % (random_state + 1, train_num))

        if train_num == 20:
            if args.dataset in ['cora', 'citeseer', 'pubmed']:
                # train_num per class: 20, val_num: 500, test: 1000
                val_num = 500
                idx_train, idx_val, idx_test = random_planetoid_splits(n_classes, torch.tensor(labels), train_num,
                                                                       random_state)
            else:
                # Coauthor CS, Amazon Computers, Amazon Photo
                # train_num per class: 20, val_num per class: 30, test: rest
                val_num = 30
                idx_train, idx_val, idx_test = get_train_data(Y, train_num, val_num, random_state)

        else:
            val_num = 0  # do not use a validation set when the training labels are extremely limited
            idx_train, idx_val, idx_test = get_train_data(Y, train_num, val_num, random_state)

        train_embs = embeds[idx_train, :]
        val_embs = embeds[idx_val, :]
        test_embs = embeds[idx_test, :]

        if train_num == 20:
            # find the best parameter C using validation set
            best_val_score = 0.0
            for param in [1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100]:
                LR = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0, C=param)
                LR.fit(train_embs, labels[idx_train])
                val_score = LR.score(val_embs, labels[idx_val])
                if val_score > best_val_score:
                    best_val_score = val_score
                    best_parameters = {'C': param}

            LR_best = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0, **best_parameters)

            LR_best.fit(train_embs, labels[idx_train])
            y_pred_test = LR_best.predict(test_embs)  # pred label
            print("Best accuracy on validation set:{:.4f}".format(best_val_score))
            print("Best parameters:{}".format(best_parameters))

        else:  # not use a validation set when the training labels are extremely limited
            LR = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0)
            LR.fit(train_embs, labels[idx_train])
            y_pred_test = LR.predict(test_embs)  # pred label

        test_acc = accuracy_score(labels[idx_test], y_pred_test)
        print("test accuaray:{:.4f}".format(test_acc))
        AccuaracyAll.append(test_acc)

    average_acc = np.mean(AccuaracyAll) * 100
    std_acc = np.std(AccuaracyAll) * 100
    print('avg accuracy over %d random splits: %.1f +/- %.1f, for train_num: %d, val_num:%d\n' % (
        numRandom, average_acc, std_acc, train_num, val_num))
    f.write('avg accuracy over %d random splits: %.1f +/- %.1f, for train_num: %d, val_num:%d\n' % (
        numRandom, average_acc, std_acc, train_num, val_num))
    f.flush()

    Accuaracy_test_allK.append(average_acc)

f.close()


Namespace(gpu=0, epochs=2000, dataset='citeseer', num_heads=4, num_layers=1, num_hidden=32, tau=1, seed=1, in_drop=0.6, attn_drop=0.5, lr=0.01, weight_decay=0.0001, negative_slope=0.2)
GATConv层输入： 3703 32 4 0.6 0.5 0.2 False <function elu at 0x000001ED01E9EE80>
num_layers: 1
Epoch 0001 | Time(s) 1.2664 | TrainLoss 8.3548 
Epoch 0002 | Time(s) 1.1998 | TrainLoss 8.1778 
Epoch 0003 | Time(s) 1.1944 | TrainLoss 8.1104 


KeyboardInterrupt: 

In [1]:
# -*- coding: utf-8 -*-
import argparse
import numpy as np
import time
import random
import torch
import torch.nn.functional as F
import dgl
from gat import GAT

from utils import load_network_data, get_train_data, random_planetoid_splits
from loss import multihead_contrastive_loss
import warnings
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser(description='GAT')
parser.add_argument("--gpu", type=int, default=0,
                    help="which GPU to use. Set -1 to use CPU.")
parser.add_argument("--epochs", type=int, default=2000,
                    help="number of training epochs")
parser.add_argument("--dataset", type=str, default="citeseer",
                    help="which dataset for training")
parser.add_argument("--num-heads", type=int, default=4,
                    help="number of hidden attention heads")
parser.add_argument("--num-layers", type=int, default=1,
                    help="number of hidden layers")
parser.add_argument("--num-hidden", type=int, default=32,
                    help="number of hidden units")
parser.add_argument("--tau", type=float, default=1,
                    help="temperature-scales")
parser.add_argument("--seed", type=int, default=1,
                    help="random seed")
parser.add_argument("--in-drop", type=float, default=0.6,
                    help="input feature dropout")
parser.add_argument("--attn-drop", type=float, default=0.5,
                    help="attention dropout")
parser.add_argument("--lr", type=float, default=0.01,
                    help="learning rate")
parser.add_argument('--weight-decay', type=float, default=1e-4,
                    help="weight decay")
parser.add_argument('--negative-slope', type=float, default=0.2,
                    help="the negative slope of leaky relu")

args = parser.parse_known_args()[0]
print(args)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

adj2, features, Y = load_network_data(args.dataset)
features[features > 0] = 1
g = dgl.from_scipy(adj2)

if args.gpu >= 0 and torch.cuda.is_available():
    cuda = True
    g = g.int().to(args.gpu)
else:
    cuda = False

features = torch.FloatTensor(features.todense())
f = open('NCLA_' + args.dataset + '.txt', 'a+')
f.write('\n\n\n{}\n'.format(args))
f.flush()

labels = np.argmax(Y, 1)
adj = torch.tensor(adj2.todense())

all_time = time.time()
num_feats = features.shape[1]
n_classes = Y.shape[1]
n_edges = g.number_of_edges()

# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# create model
heads = ([args.num_heads] * args.num_layers)
model = GAT(g,
            args.num_layers,
            num_feats,
            args.num_hidden,
            heads,
            F.elu,
            args.in_drop,
            args.attn_drop,
            args.negative_slope)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()

# use optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# initialize graph
dur = []
test_acc = 0

counter = 0
min_train_loss = 100
early_stop_counter = 100
best_t = -1

for epoch in range(1): #按照 args.epochs 指定的次数进行多轮训练。
    if epoch >= 0:
        t0 = time.time()   #计时
    model.train() #模型设置为训练模式
    optimizer.zero_grad() #梯度清零
    heads= model(features)#然后进行前向传播以获得模型预测
    loss = multihead_contrastive_loss(heads, adj, tau=args.tau)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        heads = model(features)
        loss_train = multihead_contrastive_loss(heads, adj, tau=args.tau)

    #如果训练损失连续 100 轮都没有下降，则进行提前停止
    if loss_train < min_train_loss:
        counter = 0
        min_train_loss = loss_train
        best_t = epoch
        torch.save(model.state_dict(), 'best_NCLA.pkl')
    else:
        counter += 1

    if counter >= early_stop_counter:
        print('early stop')
        break

    if epoch >= 0:
        dur.append(time.time() - t0)

    print("Epoch {:04d} | Time(s) {:.4f} | TrainLoss {:.4f} ".
          format(epoch + 1, np.mean(dur), loss_train.item()))

print('Loading {}th epoch'.format(best_t))
model.load_state_dict(torch.load('best_NCLA.pkl'))#训练结束后，从最佳轮次加载模型参数 
model.eval()#将模型设置为评估模式，并通过模型传递特征以获得嵌入
with torch.no_grad():
    heads = model(features)

Namespace(gpu=0, epochs=2000, dataset='citeseer', num_heads=4, num_layers=1, num_hidden=32, tau=1, seed=1, in_drop=0.6, attn_drop=0.5, lr=0.01, weight_decay=0.0001, negative_slope=0.2)
GATConv层输入： 3703 32 4 0.6 0.5 0.2 False <function elu at 0x0000020C82A76980>
Epoch 0001 | Time(s) 1.2394 | TrainLoss 8.3548 
Loading 0th epoch


In [ ]:
print('GATConv层输入：',in_dim, num_hidden, heads[0],
            feat_drop, attn_drop, negative_slope, False, self.activation)

In [12]:
augmented_graph_features

[tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.4445, -0.0132,  0.2077,  ..., -0.2958, -0.2379, -0.0523],
          [-0.1771, -0.5300,  0.2608,  ..., -0.3066, -0.5625,  0.2906],
          [ 0.4795, -0.3555, -0.0991,  ..., -0.2951, -0.0933, -0.5426]],
 
         [[ 0.3860, -0.1873,  0.2311,  ...,  0.3187, -0.2169, -0.3955],
          [-0.0424, -0.0533, -0.1645,  ...,  0.0020, -0.1539, -0.1314],
          [-0.1668, -0.0537, -0.0587,  ..., -0.0556, -0.1276,  0.0428],
          [ 0.0868, -0.0298, -0.0330,  ...,  0.0730, -0.1953, -0.0856]],
 
         [[-0.2173, -0.1561,  0.1978,  ...,  0.0582, -0.1489,  0.1396],
          [ 0.1280, -0.3042, -0.5491,  ...,  0.2454, -0.0274, -0.1619],
          [-0.2521,  0.0945, -0.1521,  ..., -0.0140, -0.0798,  0.0476],
          [ 0.0059, -0.1211, -0.3256,  ..., -0.0262, -0.1508, -0.1379]],
 
         ...,
 
         [[-0.0097,  0.1236, -0.0399,  ..., -0.1214,  0.1626, -0.0693],
          [ 0.0592,  0.0488, -0.2379,  

In [13]:
augmented_graph_features.shape()

AttributeError: 'list' object has no attribute 'shape'

In [14]:
type(augmented_graph_features)

list

In [15]:
my_tensor = torch.tensor(augmented_graph_features)

# 查看张量的长度


ValueError: only one element tensors can be converted to Python scalars

In [16]:
import torch

# 假设 augmented_graph_features 是一个列表，其中包含多个张量

# 使用 torch.stack() 将列表中的张量堆叠成一个新的张量
my_tensor = torch.stack(augmented_graph_features)

# 查看张量的尺寸
print("Tensor Size:", my_tensor.size())


Tensor Size: torch.Size([1, 3327, 4, 32])


In [17]:
my_tensor

tensor([[[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [-0.4445, -0.0132,  0.2077,  ..., -0.2958, -0.2379, -0.0523],
          [-0.1771, -0.5300,  0.2608,  ..., -0.3066, -0.5625,  0.2906],
          [ 0.4795, -0.3555, -0.0991,  ..., -0.2951, -0.0933, -0.5426]],

         [[ 0.3860, -0.1873,  0.2311,  ...,  0.3187, -0.2169, -0.3955],
          [-0.0424, -0.0533, -0.1645,  ...,  0.0020, -0.1539, -0.1314],
          [-0.1668, -0.0537, -0.0587,  ..., -0.0556, -0.1276,  0.0428],
          [ 0.0868, -0.0298, -0.0330,  ...,  0.0730, -0.1953, -0.0856]],

         [[-0.2173, -0.1561,  0.1978,  ...,  0.0582, -0.1489,  0.1396],
          [ 0.1280, -0.3042, -0.5491,  ...,  0.2454, -0.0274, -0.1619],
          [-0.2521,  0.0945, -0.1521,  ..., -0.0140, -0.0798,  0.0476],
          [ 0.0059, -0.1211, -0.3256,  ..., -0.0262, -0.1508, -0.1379]],

         ...,

         [[-0.0097,  0.1236, -0.0399,  ..., -0.1214,  0.1626, -0.0693],
          [ 0.0592,  0.0488, -0.2379,  ...,

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torchmeta.datasets import Cora
from torchmeta.transforms import ClassSplitter, Categorical
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.modules import MetaModule
from torchmeta.modules.utils import get_subdict

# Define a simple graph neural network model
class GraphNet(MetaModule):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphNet, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define a model that wraps the GraphNet for MAML
class MetaGraphNet(MetaModule):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MetaGraphNet, self).__init__()
        self.graphnet = GraphNet(input_dim, hidden_dim, output_dim)

    def forward(self, x, edge_index):
        return self.graphnet(x, edge_index)

# Define MAML algorithm
class MAML:
    def __init__(self, model, lr_inner=0.01, lr_outer=0.001):
        self.model = model
        self.lr_inner = lr_inner
        self.lr_outer = lr_outer
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr_outer)

    def inner_update(self, task):
        support_x, support_y = task['train']
        query_x, query_y = task['test']

        # Clone the model to initialize inner loop parameters
        inner_model = get_subdict(self.model, ['graphnet']).train()
        
        # Perform inner loop updates on support set
        for _ in range(num_inner_updates):
            logits = inner_model(support_x, task['edge_index'])
            loss = F.nll_loss(logits, support_y)
            self.model.zero_grad()
            grads = torch.autograd.grad(loss, inner_model.parameters())
            inner_model.update_params(lr=self.lr_inner, source_params=grads)

        # Compute loss on query set
        query_logits = inner_model(query_x, task['edge_index'])
        loss = F.nll_loss(query_logits, query_y)
        return loss

    def outer_update(self, tasks):
        meta_loss = torch.stack([self.inner_update(task) for task in tasks]).mean()
        self.optimizer.zero_grad()
        meta_loss.backward()
        self.optimizer.step()

# Load Cora dataset
dataset = Cora('data', num_classes_per_task=5, meta_train=True, meta_val=True, meta_test=False)
dataset = ClassSplitter(shuffle=True, num_train_per_class=1, num_test_per_class=10)(dataset)
meta_dataset = BatchMetaDataLoader(dataset, batch_size=16)

# Initialize model for MAML
model = MetaGraphNet(input_dim=dataset.num_features, hidden_dim=64, output_dim=dataset.num_classes)
maml = MAML(model)

# Train meta-learning model
num_epochs = 100
num_inner_updates = 5
for epoch in range(num_epochs):
    for tasks in meta_dataset:
        maml.outer_update(tasks)

# Get enhanced adjacency matrix and node features
enhanced_adj = []
enhanced_features = []
for task in meta_dataset:
    support_data, _ = task['train']
    enhanced_adj.append(support_data.edge_index)
    enhanced_features.append(support_data.x)
enhanced_adj = torch.stack(enhanced_adj)
enhanced_features = torch.stack(enhanced_features)

print("Enhanced adjacency matrix shape:", enhanced_adj.shape)
print("Enhanced node features shape:", enhanced_features.shape)


ModuleNotFoundError: No module named 'torchmeta'

In [1]:
# -*- coding: utf-8 -*-
import argparse
import numpy as np
import time
import random
import torch
import torch.nn.functional as F
import dgl
from gat import GAT
from utils import load_network_data, get_train_data, random_planetoid_splits
from loss import multihead_contrastive_loss
import warnings
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings("ignore")

parser = argparse.ArgumentParser(description='GAT')
parser.add_argument("--gpu", type=int, default=0,
                    help="which GPU to use. Set -1 to use CPU.")
parser.add_argument("--epochs", type=int, default=2000,
                    help="number of training epochs")
parser.add_argument("--dataset", type=str, default="cora",
                    help="which dataset for training")
parser.add_argument("--num-heads", type=int, default=4,
                    help="number of hidden attention heads")
parser.add_argument("--num-layers", type=int, default=1,
                    help="number of hidden layers")
parser.add_argument("--num-hidden", type=int, default=32,
                    help="number of hidden units")
parser.add_argument("--tau", type=float, default=1,
                    help="temperature-scales")
parser.add_argument("--seed", type=int, default=1,
                    help="random seed")
parser.add_argument("--in-drop", type=float, default=0.6,
                    help="input feature dropout")
parser.add_argument("--attn-drop", type=float, default=0.5,
                    help="attention dropout")
parser.add_argument("--lr", type=float, default=0.01,
                    help="learning rate")
parser.add_argument('--weight-decay', type=float, default=1e-4,
                    help="weight decay")
parser.add_argument('--negative-slope', type=float, default=0.2,
                    help="the negative slope of leaky relu")
args = parser.parse_known_args()[0]
print(args)

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

adj2, features, Y = load_network_data(args.dataset)
features[features > 0] = 1
g = dgl.from_scipy(adj2)

if args.gpu >= 0 and torch.cuda.is_available():
    cuda = True
    g = g.int().to(args.gpu)
else:
    cuda = False

features = torch.FloatTensor(features.todense())
f = open('NCLA_' + args.dataset + '.txt', 'a+')
f.write('\n\n\n{}\n'.format(args))
f.flush()

labels = np.argmax(Y, 1)
adj = torch.tensor(adj2.todense())

all_time = time.time()
num_feats = features.shape[1]
n_classes = Y.shape[1]
n_edges = g.number_of_edges()

# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# create model
heads = ([args.num_heads] * args.num_layers)
model = GAT(g,
            args.num_layers,
            num_feats,
            args.num_hidden,
            heads,
            F.elu,
            args.in_drop,
            args.attn_drop,
            args.negative_slope)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()

# use optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=args.lr, weight_decay=args.weight_decay)

# initialize graph
dur = []
test_acc = 0

counter = 0
min_train_loss = 100
early_stop_counter = 100
best_t = -1

for epoch in range(args.epochs):
    if epoch >= 0:
        t0 = time.time()
    model.train()
    optimizer.zero_grad()
    heads = model(features)
    loss = multihead_contrastive_loss(heads, adj, tau=args.tau)
    loss.backward()
    optimizer.step()

    model.eval()
    with torch.no_grad():
        heads = model(features)
        loss_train = multihead_contrastive_loss(heads, adj, tau=args.tau)

    # early stop if loss does not decrease for 100 consecutive epochs
    if loss_train < min_train_loss:
        counter = 0
        min_train_loss = loss_train
        best_t = epoch
        torch.save(model.state_dict(), 'best_NCLA.pkl')
    else:
        counter += 1

    if counter >= early_stop_counter:
        print('early stop')
        break

    if epoch >= 0:
        dur.append(time.time() - t0)

    print("Epoch {:04d} | Time(s) {:.4f} | TrainLoss {:.4f} ".
          format(epoch + 1, np.mean(dur), loss_train.item()))

print('Loading {}th epoch'.format(best_t))
model.load_state_dict(torch.load('best_NCLA.pkl'))
model.eval()
with torch.no_grad():
    heads = model(features)
embeds = torch.cat(heads, axis=1)  # concatenate emb learned by all heads
embeds = embeds.detach().cpu()

Accuaracy_test_allK = []
numRandom = 20

for train_num in [1, 2, 3, 4, 20]:

    AccuaracyAll = []
    for random_state in range(numRandom):
        print(
            "\n=============================%d-th random split with training num %d============================="
            % (random_state + 1, train_num))

        if train_num == 20:
            if args.dataset in ['cora', 'citeseer', 'pubmed']:
                # train_num per class: 20, val_num: 500, test: 1000
                val_num = 500
                idx_train, idx_val, idx_test = random_planetoid_splits(n_classes, torch.tensor(labels), train_num,
                                                                       random_state)
            else:
                # Coauthor CS, Amazon Computers, Amazon Photo
                # train_num per class: 20, val_num per class: 30, test: rest
                val_num = 30
                idx_train, idx_val, idx_test = get_train_data(Y, train_num, val_num, random_state)

        else:
            val_num = 0  # do not use a validation set when the training labels are extremely limited
            idx_train, idx_val, idx_test = get_train_data(Y, train_num, val_num, random_state)

        train_embs = embeds[idx_train, :]
        val_embs = embeds[idx_val, :]
        test_embs = embeds[idx_test, :]

        if train_num == 20:
            # find the best parameter C using validation set
            best_val_score = 0.0
            for param in [1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100]:
                LR = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0, C=param)
                LR.fit(train_embs, labels[idx_train])
                val_score = LR.score(val_embs, labels[idx_val])
                if val_score > best_val_score:
                    best_val_score = val_score
                    best_parameters = {'C': param}

            LR_best = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0, **best_parameters)

            LR_best.fit(train_embs, labels[idx_train])
            y_pred_test = LR_best.predict(test_embs)  # pred label
            print("Best accuracy on validation set:{:.4f}".format(best_val_score))
            print("Best parameters:{}".format(best_parameters))

        else:  # not use a validation set when the training labels are extremely limited
            LR = LogisticRegression(solver='liblinear', multi_class='ovr', random_state=0)
            LR.fit(train_embs, labels[idx_train])
            y_pred_test = LR.predict(test_embs)  # pred label

        test_acc = accuracy_score(labels[idx_test], y_pred_test)
        print("test accuaray:{:.4f}".format(test_acc))
        AccuaracyAll.append(test_acc)

    average_acc = np.mean(AccuaracyAll) * 100
    std_acc = np.std(AccuaracyAll) * 100
    print('avg accuracy over %d random splits: %.1f +/- %.1f, for train_num: %d, val_num:%d\n' % (
        numRandom, average_acc, std_acc, train_num, val_num))
    f.write('avg accuracy over %d random splits: %.1f +/- %.1f, for train_num: %d, val_num:%d\n' % (
        numRandom, average_acc, std_acc, train_num, val_num))
    f.flush()

    Accuaracy_test_allK.append(average_acc)

f.close()


Namespace(gpu=0, epochs=2000, dataset='cora', num_heads=4, num_layers=1, num_hidden=32, tau=1, seed=1, in_drop=0.6, attn_drop=0.5, lr=0.01, weight_decay=0.0001, negative_slope=0.2)
GATConv层输入： 1433 32 4 0.6 0.5 0.2 False <function elu at 0x7f798924ecb0>
num_layers: 1
NCLA当中h的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433]) <class 'torch.Tensor'>
Number of nodes: 2708
Number of edges: 13264
NCLA当中gat_layers后h的维度： torch.Size([2708, 4, 32])
NCLA当中h的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433]) <class 'torch.Tensor'>
Number of nodes: 2708
Number of edges: 13264
NCLA当中gat_layers后h的维度： torch.Size([2708, 4, 32])
Epoch 0001 | Time(s) 1.4539 | TrainLoss 8.2046 
NCLA当中h的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433])
NCLA当中temp的维度： torch.Size([2708, 1433]) <class 'torch.Tensor'>
Number of nodes: 2708
Number of edges: 13264
NCLA当中gat_layers后h的维度： torch.Si